In [10]:
!pip install tensorflow


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
#data augmentation done 
# used for building and training neural networks, including convolutional neural networks (CNNs)

In [2]:
# Simple data augmentation
data_aug = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode="nearest"
)
train_dataset = data_aug.flow_from_directory(
    "dataset//train/",
    #labels="inferred", #labels for each image is already given as it's file name
#     label_mode="binary", #binary classification
    batch_size=32,
    target_size=(224, 224), #resize of images
    class_mode="binary"
)

Found 3472 images belonging to 2 classes.


In [3]:
test_dataset = data_aug.flow_from_directory(
    "dataset//test/",
#     labels="inferred",
#     label_mode="binary",
    batch_size=32,
    target_size=(224, 224),
    class_mode="binary"
)

Found 700 images belonging to 2 classes.


In [4]:
val_dataset = data_aug.flow_from_directory(
    "dataset//val/",
#     labels="inferred",
#     label_mode="binary",
    batch_size=32,
    target_size=(224, 224),
)

Found 28 images belonging to 2 classes.


In [5]:
# Define the CNN model
#Each filter detects different features in the input image.
# Rectified Linear Unit (ReLU)
#MaxPooling helps to reduce computation and control overfitting.
# model = tf.keras.Sequential([
#     #32 is the convolutional kernels/filters
#     #3*3 is size of filter
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
#     #input_shape sets shape for first layer
#     tf.keras.layers.MaxPooling2D((2, 2)),#size of pooling window
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Flatten(),#flattens the 2D feature maps into a 1D vector.
#     tf.keras.layers.Dense(128, activation='relu'),#learns patterns in flattened data
#     tf.keras.layers.Dense(1, activation='sigmoid'),
# ])


resnet_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))



In [6]:
for layer in resnet_model.layers:
  layer.trainable = False
custom_head = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model = tf.keras.Sequential([resnet_model, custom_head])


In [7]:
# Compile the model
#adam is optimization algorithm
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])


model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])



In [8]:
# Train the model
model.fit(train_dataset, epochs=10)

Epoch 1/10


109/109 [==============================] - 371s 3s/step - loss: 0.1445 - accuracy: 0.9510
Epoch 2/10
109/109 [==============================] - 246s 2s/step - loss: 0.0748 - accuracy: 0.9761
Epoch 3/10
109/109 [==============================] - 224s 2s/step - loss: 0.0618 - accuracy: 0.9772
Epoch 4/10
109/109 [==============================] - 193s 2s/step - loss: 0.0525 - accuracy: 0.9827
Epoch 5/10
109/109 [==============================] - 218s 2s/step - loss: 0.0469 - accuracy: 0.9850
Epoch 6/10
109/109 [==============================] - 199s 2s/step - loss: 0.0401 - accuracy: 0.9879
Epoch 7/10
109/109 [==============================] - 205s 2s/step - loss: 0.0342 - accuracy: 0.9893
Epoch 8/10
109/109 [==============================] - 174s 2s/step - loss: 0.0363 - accuracy: 0.9879
Epoch 9/10
109/109 [==============================] - 173s 2s/step - loss: 0.0375 - accuracy: 0.9868
Epoch 10/10
109/109 [==============================] - 171s 2s/step - loss: 0.0364 - accu

In [9]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_dataset)

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

# Save the model
model.save('CNN_prediction_model.keras')
# Test loss: 0.08772128075361252
# Test accuracy: 0.9642857313156128

22/22 [==============================] - 40s 2s/step - loss: 0.1279 - accuracy: 0.9457
Test loss: 0.12793955206871033
Test accuracy: 0.9457142949104309


In [20]:
image_path = '../unseenTB2.png'
def predict_tb_with_cnn(image_path):
    # Load the pre-trained CNN model
#     model = tf.keras.models.load_model('CNN_prediction_model.keras')

    # Load and preprocess the image
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = tf.expand_dims(image, axis=0)

    # Predict the probability of tuberculosis
    prediction = model.predict(image)

    # Get the probability of tuberculosis
    probability_of_tb = prediction[0][0]

    # Convert the probability to a binary output
    if probability_of_tb > 0.5:
        output = 'TB'
    else:
        output = 'Normal'
    print(output)
    return output

# model.save('CNN_prediction_model.keras')

In [22]:
# Load the X-ray image
image = tf.keras.preprocessing.image.load_img('../unseenTB2.png', target_size=(224, 224))
#wrong output - 202,206

In [23]:
# Preprocess the image
image = tf.keras.preprocessing.image.img_to_array(image)
image = tf.expand_dims(image, axis=0)

In [25]:
# Make the prediction
prediction = model.predict(image)

# Get the probability of tuberculosis
probability_of_tb = prediction[0][0]

print(probability_of_tb)
# Convert the probability to a binary output
if probability_of_tb > 0.5:
    output = 'TB'
else:
    output = 'Normal'

# Print the output
print('Output:', output)

1/1 [==============================] - 0s 115ms/step
0.23192726
1/1 [==============================] - 0s 134ms/step
Normal
Output: Normal
